<a href="https://colab.research.google.com/github/MaFiN1337/crime-stats-ua/blob/main/generate_charts_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# generate_charts.py


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import plotly.express as px
import os

DATA_FILE = '/content/drive/MyDrive/DataSet_DaniTaSusp.xlsx'

OUTPUT_DIR = 'charts'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

try:
    df = pd.read_excel(DATA_FILE)
except FileNotFoundError:
    print(f"ПОМИЛКА: Файл не знайдено за шляхом {DATA_FILE}")
    print("Будь ласка, перевірте шлях і завантажте файл.")
    raise SystemExit

df['Date'] = pd.to_datetime(
    df['Year'].astype(str) + '-' + df['Month'].astype(str) + '-01',
    errors='coerce'
)
df = df.dropna(subset=['Date'])
df = df.sort_values(by='Date')

articles_to_plot = df['Article'].unique()
years_to_plot = df['Year'].unique()

metrics_to_plot = [
    'Oblikovano_total',
    'Pidozra',
    'Zupineno',
    'Napravleno_do_sudu',
    'Provadjenya_zakryto',
    'Ne_priynyato_rishennya'
]

print(f"Знайдено статті: {articles_to_plot}")
print(f"Знайдено роки: {years_to_plot}")
print(f"Знайдено показники: {metrics_to_plot}")

total_charts = 0

print("Генеруємо річні (помісячні) графіки...")
for metric in metrics_to_plot:
    for article in articles_to_plot:
        for year in years_to_plot:

            data_subset = df[
                (df['Article'] == article) &
                (df['Year'] == year)
            ]

            if data_subset.empty:
                continue

            fig = px.bar(
                data_subset,
                x='Month',
                y=metric,
                title=f'Показник "{metric}" (помісячно)<br>Стаття {article}, {year} рік',
                labels={'Month': 'Місяць', metric: 'Кількість'}
            )
            fig.update_layout(xaxis=dict(tickmode='linear', dtick=1))

            filename = f"monthly_{article}_{year}_{metric}.html"
            save_path = os.path.join(OUTPUT_DIR, filename)

            fig.write_html(save_path)
            total_charts += 1

print(f"Згенеровано {total_charts} річних графіків.")
print("Генеруємо загальні (часові) графіки...")
for metric in metrics_to_plot:
    for article in articles_to_plot:

        data_subset = df[df['Article'] == article]

        if data_subset.empty:
            continue

        fig = px.line(
            data_subset,
            x='Date',
            y=metric,
            title=f'Загальна динаміка "{metric}"<br>Стаття {article} (2014-2024)',
            labels={'Date': 'Дата', metric: 'Кількість'},
            markers=True
        )

        filename = f"timeseries_{article}_{metric}.html"
        save_path = os.path.join(OUTPUT_DIR, filename)

        fig.write_html(save_path)
        total_charts += 1

print(f"УСЬОГО згенеровано {total_charts} HTML файлів у папці '{OUTPUT_DIR}'.")

Знайдено статті: [111 121 185 187 190]
Знайдено роки: [2014 2015 2016 2017 2018 2019 2020 2021 2022 2023 2024]
Знайдено показники: ['Oblikovano_total', 'Pidozra', 'Zupineno', 'Napravleno_do_sudu', 'Provadjenya_zakryto', 'Ne_priynyato_rishennya']
Генеруємо річні (помісячні) графіки...
Згенеровано 330 річних графіків.
Генеруємо загальні (часові) графіки...
УСЬОГО згенеровано 360 HTML файлів у папці 'charts'.
